In [1]:
# Import Libraries, Census, and Keys
import pandas as pd
import requests
from census import Census
import gmaps
import json
from config import (api_key, gkey)
c = Census(api_key, year=2017)

# Configure gmaps (not sure why we need this)
gmaps.configure(api_key=gkey)


In [2]:
# Search Census data, retrive relavant information by state
census_data = c.acs5.get(("NAME", "B01003_001E", "B19301_001E", 
                          "B17001_002E","B17001A_002E","B17001B_002E",
                          "B17001D_002E","B17001I_002E"),{'for': 'state:*'})

# Make DF
census_df = pd.DataFrame(census_data)

# Rename to make user friendly
census_df = census_df.rename(columns={"B01003_001E": "Population",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Total Poverty",
                                      "B17001A_002E": "White Poverty",
                                      "B17001B_002E": "Black Poverty",
                                      "B17001D_002E": "Asian Poverty",
                                      "B17001I_002E": "Hispanic Poverty",
                                      "NAME": "Name", 
                                      "state": "State"})

# Rearrange DF
census_df = census_df[["State", "Name","Population", "Per Capita Income",
                       "Per Capita Income", "Total Poverty", "Total Poverty",
                       "White Poverty", "Black Poverty", "Asian Poverty",
                       "Hispanic Poverty"]]

# See my data
census_df.head()

,State,Name,Population,Per Capita Income,Per Capita Income,Total Poverty,Total Poverty,White Poverty,Black Poverty,Asian Poverty,Hispanic Poverty
0,72,Puerto Rico,3468963.0,12081.0,12081.0,1543220.0,1543220.0,1026976.0,154131.0,4113.0,1530395.0
1,01,Alabama,4850771.0,25746.0,25746.0,849699.0,849699.0,432404.0,360407.0,8597.0,62607.0
2,02,Alaska,738565.0,35065.0,35065.0,73380.0,73380.0,31659.0,3387.0,4673.0,5463.0
3,04,Arizona,6809946.0,27964.0,27964.0,1128046.0,1128046.0,764158.0,62682.0,27329.0,513987.0
4,05,Arkansas,2977944.0,24426.0,24426.0,524211.0,524211.0,332449.0,139406.0,5909.0,61991.0


In [3]:
# Set parameters 
params = {"key":gkey}
base_url = "https://maps.googleapis.com/maps/api/geocode/json"

# Run through all rows in my DF
for index, row in census_df.iterrows():
    
    # Grab the "Name" column of every Index/Row
    state_type = row['Name']
    
    # Add newly grabbed data and put in inside "params" dictionary
    params['address'] = state_type 
    
    # Now that paramerts are complete, read URL with json
    response = requests.get(base_url, params = params).json()
    
    # Take care of any possible issues
    try:
        
        # After digging through URL data, set desired info equal to "lat" & "lng"
        latitude = response['results'][0]["geometry"]['location']['lat']
        longitude = response['results'][0]["geometry"]['location']['lng'] 
        
        # Put data into DF
        census_df.loc[index,'Latitude']= latitude
        census_df.loc[index,'Longitude']= longitude
        
    # Possible issues with code
    except (KeyError, IndexError):
        print(f'Not Found')
        
# See my Data
census_df.head(10)


,State,Name,Population,Per Capita Income,Per Capita Income,Total Poverty,Total Poverty,White Poverty,Black Poverty,Asian Poverty,Hispanic Poverty,Latitude,Longitude
0,72,Puerto Rico,3468963.0,12081.0,12081.0,1543220.0,1543220.0,1026976.0,154131.0,4113.0,1530395.0,18.220833,-66.590149
1,01,Alabama,4850771.0,25746.0,25746.0,849699.0,849699.0,432404.0,360407.0,8597.0,62607.0,32.318231,-86.902298
2,02,Alaska,738565.0,35065.0,35065.0,73380.0,73380.0,31659.0,3387.0,4673.0,5463.0,64.200841,-149.493673
3,04,Arizona,6809946.0,27964.0,27964.0,1128046.0,1128046.0,764158.0,62682.0,27329.0,513987.0,34.048928,-111.093731
4,05,Arkansas,2977944.0,24426.0,24426.0,524211.0,524211.0,332449.0,139406.0,5909.0,61991.0,35.201050,-91.831833
5,06,California,38982847.0,33128.0,33128.0,5773408.0,5773408.0,3183011.0,502610.0,607792.0,3052999.0,36.778261,-119.417932
6,08,Colorado,5436519.0,34845.0,34845.0,612143.0,612143.0,467164.0,41732.0,18258.0,218275.0,39.550051,-105.782067
7,09,Connecticut,3594478.0,41365.0,41365.0,350783.0,350783.0,205283.0,66875.0,12665.0,123584.0,41.603221,-73.087749
8,11,District of Columbia,672391.0,50832.0,50832.0,111025.0,111025.0,18242.0,81287.0,3697.0,9665.0,38.907192,-77.036871
9,10,Delaware,943732.0,32625.0,32625.0,111304.0,111304.0,59997.0,37039.0,2983.0,20127.0,38.910832,-75.527670


In [3]:
#hispanic_poverty = censuus_df['Hispanic Poverty']

NameError: name 'censuus_df' is not defined